In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

# 设置并固定随机种子

In [2]:
# 设置随机种子
np.random.seed(42)
random_state = 42

# 加载并清洗数据

In [65]:
# 1. 数据预处理
# 加载训练数据集（假设训练数据集文件名为 iris_train.csv）
train_df = pd.read_csv('./data/iris_train.csv')

# 数据清洗：检查缺失值和异常值
# 鸢尾花数据集通常没有缺失值和异常值，这里仅作为示例
train_df_cleaned = train_df.dropna()  # 删除缺失值

# 特征选择：选择所有特征
X_train = train_df_cleaned.drop(columns=['Species'])  # 标签列名为 'species'
y_train = train_df_cleaned['Species']

# 数据标准化/归一化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


# 保存清洗后的训练数据到指定文件 

In [66]:

preprocessed_train_data = pd.concat([pd.DataFrame(X_train_scaled, columns=X_train.columns), pd.Series(y_train, name='species')], axis=1)
preprocessed_train_data.to_csv('./output/cleaned_train_data.csv', index=False)


# 训练标签转换

In [67]:
# 将字符串标签转换为整数标签
le = LabelEncoder()
y_train_int = le.fit_transform(y_train)

# 反转
y_train_int_reverse = le.inverse_transform(y_train_int)
print(all(y_train_int_reverse==y_train))

True


# 数据打乱

In [68]:
# 打乱数据（可选，但在使用随机种子时可以确保一致性）
X_train_scaled_s, y_train_int_s = shuffle(X_train_scaled, y_train_int, random_state=random_state)

# 模型选择

In [ ]:
# import h2o
# from h2o.automl import H2OAutoML
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import pandas as pd

# # 初始化 H2O
# h2o.init()

# # # 加载鸢尾花数据集
# # data = load_iris()
# # X, y_str = data.data, data.target_names[data.target]  # 获取原始标签名

# # # 将字符串标签转换为整数标签
# # le = LabelEncoder()
# # y_int = le.fit_transform(y_str)

# # 将数据转换为 DataFrame
# # df = pd.DataFrame(X_train, columns=data.feature_names)
# df = X_train.copy(deep=True)
# df['target'] = y_train_int

# # 将 DataFrame 转换为 H2O Frame
# hf = h2o.H2OFrame(df)

# # 指定目标列
# y = 'target'
# x = hf.columns
# x.remove(y)

# # 划分训练集和测试集
# train, test = hf.split_frame(ratios=[0.9], seed=42)

# # --------------------------
# # 测试集
# # 加载验证数据集（假设验证数据集文件名为 iris_val.csv）
# val_df = pd.read_csv('./data/iris_test.csv')

# # 数据清洗：检查缺失值和异常值
# val_df_cleaned = val_df.dropna()  # 删除缺失值

# # 特征选择：选择所有特征
# X_val = val_df_cleaned.drop(columns=['Species'])  # 标签列名为 'species'

# # 数据标准化/归一化
# X_val_scaled = scaler.transform(X_val)

# # --------------------------


# # 创建 AutoML 实例
# aml = H2OAutoML(max_runtime_secs=120,  # 设置 AutoML 运行的最大时间（秒）
#                 seed=42,               # 设置随机种子以确保结果可复现
#                 nfolds=5,              # 交叉验证的折数
#                 sort_metric='accuracy')  # 选择评估指标

# # 训练 AutoML 模型
# aml.train(x=x, y=y, training_frame=train)

# # 查看最佳模型
# leader_model = aml.leader
# print("Best model found by H2O AutoML:")
# print(leader_model)

# # 查看最佳模型的性能
# performance = leader_model.model_performance(test_data=test)
# print("Performance on the test set:")
# print(performance)

# # 如果需要，可以使用最佳模型进行预测
# # predictions = leader_model.predict(test)

MLP: 0.9750
Gradient Boosting: 0.9583
XGBoost: 0.9583
Random Forest: 0.9500
SVM: 0.9500
Naive Bayes: 0.9500
Logistic Regression: 0.9417
Decision Tree: 0.9417
K-Nearest Neighbors: 0.9167

The best model is MLP with an average accuracy of 0.9750


# 模型选择2

In [3]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# 初始化 H2O
h2o.init()

# 加载训练数据
train_df = pd.read_csv('iris_train.csv')

# 将训练数据转换为 H2O Frame
train_hf = h2o.H2OFrame(train_df)

# 指定目标列
y = 'Species'  # 假设标签列名为 'Species'
x = train_hf.columns
x.remove(y)

# 加载测试数据
test_df = pd.read_csv('iris_test.csv')

# 将测试数据转换为 H2O Frame
test_hf = h2o.H2OFrame(test_df)

# 创建 AutoML 实例
aml = H2OAutoML(max_runtime_secs=120,  # 设置 AutoML 运行的最大时间（秒）
                seed=42,               # 设置随机种子以确保结果可复现
                nfolds=5,              # 交叉验证的折数
                sort_metric='accuracy')  # 选择评估指标

# 训练 AutoML 模型
aml.train(x=x, y=y, training_frame=train_hf)

# 查看最佳模型
leader_model = aml.leader
print("Best model found by H2O AutoML:")
print(leader_model)

# 使用最佳模型在测试集上进行预测
predictions = leader_model.predict(test_hf)

# 将预测结果转换为 Pandas DataFrame
predictions_df = predictions.as_data_frame()

# 将测试集的真实标签转换为 Pandas DataFrame
true_labels_df = test_hf[y].as_data_frame()

# 计算测试集上的准确率
accuracy = accuracy_score(true_labels_df, predictions_df['predict'])
print(f"Accuracy on the test set: {accuracy:.4f}")

# 将预测结果保存到 CSV 文件
predictions_df.to_csv('iris_test_predictions.csv', index=False)

# 打印预测结果
print("Predictions on the test set:")
print(predictions_df.head())

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...


H2OStartupError: Cannot find Java. Please install the latest JRE from
http://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html#java-requirements

# 某次“模型选择”输出的结果

# 模型训练
> 数据分类在数据挖掘中是一种常见场景，您可以尝试使用不同建模方式(决策树模型,随机森林，SVC。。。。。。)

MLPClassifier(max_iter=1000, random_state=42)

训练集准确率: 0.99


In [85]:
# 3. 模型推理
# 加载验证数据集（假设验证数据集文件名为 iris_val.csv）
val_df = pd.read_csv('./data/iris_test.csv')

# 数据清洗：检查缺失值和异常值
val_df_cleaned = val_df.dropna()  # 删除缺失值

# 特征选择：选择所有特征
X_val = val_df_cleaned.drop(columns=['Species'])  # 标签列名为 'species'

# 数据标准化/归一化
X_val_scaled = scaler.transform(X_val)

# 模型推理
y_pred = model.predict(X_val_scaled)

# 保存推理结果到指定目录(确保文件名，字段名正确)

In [86]:
# 保存推理结果
results = pd.DataFrame({'Sample_ID': range(len(y_pred)), 'Predicted_Species': y_pred})
results.to_csv('./output/test_data_predictions.csv', index=False)

# 试试检查您训练的结果
> 真实竞赛中会根据模型准确率评分，所以提供的推理数据不会包含标签列验证,您竞赛中不需要完成下面步骤

In [87]:
val_df_cleaned['pred']=y_pred
val_df_cleaned[['Species','pred']]
# 提取验证集的真实标签和预测结果
y_true = val_df_cleaned['Species']
y_pred = val_df_cleaned['pred']

## NOTE: 转化
y_true_int = le.transform(y_true)

# 评估模型
# accuracy = accuracy_score(y_true, y_pred)
# print(f"模型准确率: {accuracy:.2f}")
# print("分类报告:")
# print(classification_report(y_true, y_pred))

accuracy = accuracy_score(y_true_int, y_pred)
print(f"模型准确率: {accuracy:.2f}")
print("分类报告:")
print(classification_report(y_true_int, y_pred))

模型准确率: 0.93
分类报告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.85      1.00      0.92        11
           2       1.00      0.78      0.88         9

    accuracy                           0.93        30
   macro avg       0.95      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



# <font color="red">最后您需要将这个notebook保存到output目录(可以通过左边的文件管理栏,或文件菜单中的另存为)，供评分使用<font color="red">